In [13]:
#hide
#default_exp tools.copy_experiment
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Copy experiment indicated by argument

In [14]:
#export
import argparse
import sys
import inspect
import shutil
import joblib
import os
import re
from distutils.dir_util import copy_tree

from hpsearch.experiment_manager import mypprint
from hpsearch.config.hpconfig import get_experiment_manager
from hpsearch.tools.query import query
import hpsearch.config.hp_defaults as dflt

In [15]:
#for tests
import pytest
import os
from hpsearch.examples.complex_dummy_experiment_manager import generate_data, init_em

## get_best_experiment

```python
#export
def get_best_experiment (root=None, metric=None, runs=None, op=None, 
                         stats=None, results=None, other_parameters={}, 
                         sort=None, show=False):
    df = query (root=root,
               metric=metric, runs=runs, op=op, stats=stats,
               results=results, other_parameters=other_parameters)
    if sort is not None:
        assert sort in df.columns, ('sort must be a column in '
                                    f'dataframe ({df.columns})')
        df = df.sort_values(by=sort, ascending=(op=='min'))
    best = df.index[0]
    if show:
        display (df)
    return best
```

## copy_experiment_contents

In [16]:
#export
def copy_experiment_contents (experiment=None, root=None, destination_folder='.', 
                              run=0, desired=None, target_model=None,
                              destination_model=None, manager_path=dflt.manager_path):
    
    os.makedirs (destination_folder, exist_ok=True)
    
    em = get_experiment_manager (manager_path=manager_path)
    
    # 3 write code about calling run_experiment
    root_path = em.get_path_experiments(folder=root)
    path_root_experiment = em.get_path_experiment (experiment, root_path=root_path)
    shutil.copy (f'{path_root_experiment}/parameters.pk', 
                 f'{destination_folder}/separate_parameters.pk')
    path_results = em.get_path_results (experiment, run, root_folder=root)
    if desired is not None and 'path_results' in desired:
        assert path_results == desired['path_results']
    copy_tree (f'{path_results}', destination_folder)
    target_model = (target_model if target_model is not None 
                    else em.target_model_file)
    destination_model = (destination_model if destination_model is not None 
                    else em.destination_model_file)
    if target_model is not None and destination_model is not None:
        shutil.copy (f'{path_results}/{target_model}', 
                     f'{destination_folder}/{destination_model}')

## copy_code

In [17]:
#export
def copy_code (source_folder, destination_folder, file=None,
               path_root_experiment=None, manager_path=dflt.manager_path):
    
    os.makedirs (destination_folder, exist_ok=True)
    destination_path = (f'{destination_folder}/best_experiment.py' if file is None 
                        else f'{destination_folder}/{file}')
    path_root_experiment = (source_folder if path_root_experiment is None
                            else path_root_experiment)
    path_results = source_folder
    fdest = open (destination_path, 'wt')
    
    # 1 write code before definition of subclassed manager
    em = get_experiment_manager (manager_path=manager_path)
    source_path = inspect.getfile(em.__class__)
    fsrc = open (source_path, 'rt')
    original = fsrc.read ()
    fsrc.close ()
    original.replace ('from hpsearch.experiment_manager import ExperimentManager', '')
    
    # 2 write code from base manager
    try:
        r = re.findall (f'class\s+{em.__class__.__name__}\s*\(.*\)', original)
        assert len(r) == 1, 'class not found'
        name_base_class = re.findall ('\(.*\)', r[0])[0][1:-1]
    except Exception as e:
        print (f'{e}')
        name_base_class = 'ExperimentManager'

    base_manager = f'class {name_base_class} ():'
    base_manager = base_manager + '''
    def __init__ (self, *args, **kwargs):
        pass
'''
    r= re.findall (f'class\s+{em.__class__.__name__}', original)
    assert len(r) == 1, 'class not found'
    r = original.find (r[0])
    original = (original[:r-1] + base_manager + '\n' + original[r:])
    fdest.write (original)
    
    # 3 write code about calling run_experiment
    try:
        parameters, other_parameters, *_ = joblib.load (f'{path_root_experiment}/separate_parameters.pk')
    except FileNotFoundError:
        parameters, other_parameters, *_ = joblib.load (f'{path_root_experiment}/parameters.pk')
    parameters = mypprint(parameters, dict_name='    parameters')
    
    path_results_str = '{path_results}'
    run_experiment = (
f'''
def main (**kwargs):
    path_results = {path_results}
    all_parameters = joblib.load (f'{path_results_str}/parameters.pk')
{parameters}
    all_parameters.update (parameters)
    all_parameters.update (**kwargs)
    em = {em.__class__.__name__} ()
    em.run_experiment (all_parameters, path_results)

if __name__ == '__main__':
    main ()
'''
    )
    fdest.write (run_experiment)
    fdest.close ()

## copy_experiment_contents_and_code

In [18]:
#export
def copy_experiment_contents_and_code (experiment=None, root=None, 
                                       content='.', 
                                       code='.',
                                       run=0, 
                                       file=None, 
                                       target_model=None,
                                       destination_model=None,
                                       desired=None,
                                       manager_path=dflt.manager_path):
    copy_experiment_contents (experiment=experiment, root=root, 
                              destination_folder=content, 
                              run=run, target_model=target_model,
                              destination_model=destination_model,
                              desired=desired, manager_path=manager_path)
    copy_code (content, code, file=file,
               path_root_experiment=content, 
               manager_path=manager_path)

### Usage example

In [19]:
#export tests.tools.test_copy_experiment
def test_copy_content_and_code ():
    em = generate_data ('copy_experiment_contents')

    # function
    copy_experiment_contents_and_code (
        experiment=2, content='test_dest_folder_copy_exp_content', 
        code='test_my_code', run=0, file='test_file.py',
        manager_path=em.manager_path
    )

    assert sorted(os.listdir ('test_dest_folder_copy_exp_content'))==[
     'dict_results.pk', 'dummy_experiment_manager.py', 'logs.txt', 'model_history.pk',
     'model_weights.pk', 'parameters.json', 'parameters.pk',
     'parameters.txt', 'separate_parameters.pk', 'summary.txt']
    
    shutil.rmtree ('test_dest_folder_copy_exp_content')
    copy_experiment_contents_and_code (
        experiment=2, content='test_dest_folder_copy_exp_content', 
        code='test_my_code', run=0, file='test_file.py',
        target_model='model_weights.pk', destination_model='nn_model.pk',
        manager_path=em.manager_path
    )
    assert sorted(os.listdir ('test_dest_folder_copy_exp_content')) == [
        'dict_results.pk', 'dummy_experiment_manager.py', 'logs.txt', 'model_history.pk', 
        'model_weights.pk', 'nn_model.pk', 'parameters.json', 
        'parameters.pk', 'parameters.txt', 'separate_parameters.pk', 
        'summary.txt']
    
    # checks
    assert os.listdir ('test_my_code')==['test_file.py']
    f = open (f'test_my_code/test_file.py', 'rt')
    text = f.read()
    f.close()
    assert 'ExperimentManager ()' in text
    assert 'parameters=dict(epochs=5,' in text
    assert 'em = ComplexDummyExperimentManager ()' in text
    
    shutil.rmtree ('test_my_code')
    shutil.rmtree ('test_dest_folder_copy_exp_content')
    em.remove_previous_experiments ()

In [20]:
tst.run (test_copy_content_and_code, tag='dummy')

running test_copy_content_and_code
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error

## copy_code_with_experiment_paths

In [21]:
#export
def copy_code_with_experiment_paths (experiment=None, root=None, 
                                     destination_folder='.', file=None,
                                     run=0, desired=None, manager_path=dflt.manager_path):
    em = get_experiment_manager (manager_path=manager_path)
    root_path = em.get_path_experiments(folder=root)
    path_root_experiment = em.get_path_experiment (experiment, root_path=root_path)
    path_results = em.get_path_results (experiment, run, root_folder=root)
    copy_code (path_results, destination_folder, file=file,
               path_root_experiment=path_root_experiment,
               manager_path=manager_path)
    

### Usage example

In [22]:
#export tests.tools.test_copy_experiment
def test_copy_code_with_experiment_paths ():
    em = generate_data ('copy_code_with_experiment_paths')
    
    # function
    copy_code_with_experiment_paths (experiment=2, 
                                     destination_folder='test_dest_folder_copy_exp', 
                                     file='test_file.py', run=0,
                                     manager_path=em.manager_path)
    
    # checks
    assert os.listdir ('test_dest_folder_copy_exp')==['test_file.py']
    f = open (f'test_dest_folder_copy_exp/test_file.py', 'rt')
    text = f.read()
    f.close()
    assert 'ExperimentManager ()' in text
    assert 'parameters=dict(epochs=5,' in text
    assert 'em = ComplexDummyExperimentManager ()' in text
    
    em.remove_previous_experiments ()
    shutil.rmtree ('test_dest_folder_copy_exp')

In [23]:
tst.run (test_copy_code_with_experiment_paths, tag='dummy')

running test_copy_code_with_experiment_paths
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parameters
error saving other parame

## parse_arguments_and_run

In [24]:
#export
def parse_args (args):
    parser = argparse.ArgumentParser(description='copy experiment')
    parser.add_argument('-e', '--experiment', type=int, default=None,  
                        help="experiment number")
    parser.add_argument('--run', type=int, default=None,  help="run number")
    parser.add_argument('--root', type=str, default=None, help='name of root folder')
    parser.add_argument('--content', type=str, 
                        default='.', help='path to folder containing experiment results')
    parser.add_argument('--code', type=str, 
                        default='.', help='path to folder containing experiment results')
    parser.add_argument('--file', type=str, default=None, help='destination code file')
    parser.add_argument('--target_model', type=str, default=None, 
                        help='target model file')
    parser.add_argument('--destination_model', type=str, default=None, 
                        help='destination model file')
    parser.add_argument('-p', '--path', default=dflt.manager_path, type=str)
    
    pars = parser.parse_args(args)
    
    return pars

def parse_arguments_and_run (args, desired = None):
    
    pars = parse_args(args)
    pars = vars(pars)
    pars['manager_path'] = pars['path']
    del pars['path']
    copy_experiment_contents_and_code (**pars, desired=desired)

def main():
    parse_arguments_and_run (sys.argv[1:])

### Usage example

In [25]:
#export tests.tools.test_copy_experiment
def test_parse_arguments_copy_experiment ():
    em = generate_data ('parse_arguments_copy_experiment', root_folder='newroot')
    
    df = em.get_experiment_data (folder_experiments='newroot')
    assert df.shape==(9,25)
        
    args = ('--root newroot -e 2 --content test_dest_folder_copy_exp_content '
           f'--run 1 --file test_file.py --code test_my_code -p {em.manager_path}')
    desired = {'path_results': 
               'test_parse_arguments_copy_experiment/newroot/experiments/00002/1'}
    parse_arguments_and_run (args.split (), desired=desired)
    
    assert sorted(os.listdir ('test_dest_folder_copy_exp_content'))==[
     'dict_results.pk',
     'dummy_experiment_manager.py',
     'logs.txt',
     'model_history.pk',
     'model_weights.pk',
     'parameters.json',
     'parameters.pk',
     'parameters.txt',
     'separate_parameters.pk',
     'summary.txt']
    
    # checks
    assert os.listdir ('test_my_code')==['test_file.py']
    f = open (f'test_my_code/test_file.py', 'rt')
    text = f.read()
    f.close()
    assert 'ExperimentManager ()' in text
    assert 'parameters=dict(epochs=5,' in text
    assert 'em = ComplexDummyExperimentManager ()' in text
    
    shutil.rmtree ('test_my_code')
    shutil.rmtree ('test_dest_folder_copy_exp_content')
    em.remove_previous_experiments ()

In [26]:
tst.run (test_parse_arguments_copy_experiment, tag='dummy', debug=False)

running test_parse_arguments_copy_experiment


ValueError: self.root != self.root_folder

In [27]:
%debug

> /home/jcidatascience/jaume/workspace/remote/hpsearch/hpsearch/experiment_manager.py(101)__init__()
     99             self.root = self.root_folder
    100         if self.root is not None and self.root_folder is not None and self.root != self.root_folder:
--> 101             raise ValueError ('self.root != self.root_folder')
    102 
    103         self.registered_name = (f'{class_name}-default' if (self.root_folder is None



ipdb>  self.root


''


ipdb>  self.root_folder


'newroot'


ipdb>  dflt.root


''


ipdb>  q
